In [45]:
# Install the library using pip
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import os
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from sentence_transformers import SentenceTransformer
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from transformers import pipeline
from sentence_transformers import util

In [47]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:

# Load the trained image classification model
model_path = '/content/drive/MyDrive/apple_resnet_classifier.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(model_path, map_location=device)
model.to(device)
model.eval()

# Define the image transformations
transform_img_normal = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Specify the folder location
folder_url = "/content/drive/MyDrive/appel_crop_selectnew/Train"

# Load the dataset
dataset = ImageFolder(folder_url, transform=transform_img_normal)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=50, shuffle=False)

In [51]:
# Function to predict the class labels
def predict(model, data):
    class_labels = ['Bad Apple', 'Normal Apple', 'Rot Apple', 'Scab Apple']
    class_counts = [0, 0, 0, 0]

    with torch.no_grad():
        for inputs, labels in data:
            inputs = inputs.to(device)
            out = model(inputs)
            _, predicted = torch.max(out.data, 1)

            for p in predicted:
                class_counts[p.item()] += 1

    label_counts_dict = {label: count for label, count in zip(class_labels, class_counts)}
    return label_counts_dict

# Perform prediction on the dataset
apple_counts = predict(model, dataset_loader)

# Get the apple counts from the prediction result
apple_counts = list(apple_counts.values())

# Specify the desired sample size
sample_size = int(input('Enter the sample size: '))

# Randomly select test apples from available_apples
random_test_apples = random.choices(apple_counts, k=sample_size)

print('Randomly selected apples:', random_test_apples)

Enter the sample size: 30
Randomly selected apples: [137, 137, 62, 109, 109, 62, 62, 62, 62, 137, 80, 80, 137, 137, 109, 109, 137, 80, 137, 109, 109, 80, 62, 80, 80, 109, 80, 80, 62, 137]


In [52]:
# Create the training data for AQL class prediction
mylist = [
    'aql_calculation',
    'class_determination',
    'The class of apples is based on the number of apples in each category.',
    "AQL(Acceptable Quality Limit) of class 1 is 0.4 or less than 0.4 and it can be accepted as class 1 if n(batch size) = 32 and x(the number of not Normal Apple) < 1.",
    "AQL of class 2 is more than 0.4 and less than 6.5 or equal to 6.5 and it can be accepted as class 2 if n = 20 and x < 8.",
    "AQL of class 3 is more than 6.5 and less than 15 or equal to 15 and it can be accepted as class 3 if n = 20 and x < 15.",
    "AQL of class 4 is more than 15 and it can be class 4 if n = 20 and x >= 15.",
    "The AQL class depends on the batch size and the number of not Normal Apples.",
    "To determine the AQL class, compare the batch size (n) and the number of not Normal Apples (x) with the specified thresholds.",
]

X_train = mylist[:-4] + [mylist[-3]] + [mylist[-2]] + [mylist[-1]]  # Include the additional samples
y_train = ['class 1', 'class 2', 'class 3', 'class 4', 'Unknown', 'Unknown', 'Unknown']  # Define the corresponding classes

# Filter out the unknown classes
X_train_filtered = [x for x, y in zip(X_train, y_train) if y in ['class 1', 'class 2', 'class 3', 'class 4']]
y_train_filtered = [y for y in y_train if y in ['class 1', 'class 2', 'class 3', 'class 4']]

# Train a LinearSVC model using TF-IDF features
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_filtered)
model = LinearSVC()
model.fit(X_train_vectorized, y_train_filtered)

# Use SentenceTransformer for text similarity
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to predict the AQL class based on the input text
def predict_aql_class(input_text):
    # Generate embeddings for the training data and the input text
    X_train_embeddings = sbert_model.encode(X_train)
    input_embedding = sbert_model.encode([input_text])

    # Calculate the cosine similarities between the input embedding and the training embeddings
    cosine_similarities = util.pytorch_cos_sim(input_embedding, torch.tensor(X_train_embeddings))
    # Find the most similar training passage
    most_similar_index = cosine_similarities.argmax().item()

    # Predict the AQL class based on the index
    predicted_class = y_train[most_similar_index]

    return predicted_class



In [53]:
# Function to calculate the AQL class based on the given batch size (n) and number of not Normal Apples (x)
def calculate_aql_class(n, x):
    if n == 32 and x < 1:
        return 'class 1'
    elif n == 20 and x < 8:
        return 'class 2'
    elif n == 20 and x < 15:
        return 'class 3'
    elif n == 20 and x >= 15:
        return 'class 4'
    else:
        return 'Unknown'

# Get the number of files in the "Normal_Apple" folder
normal_apples_folder = "/content/drive/MyDrive/appel_crop_selectnew/Train/Normal_Apple"
normal_apples_count = len(os.listdir(normal_apples_folder))


In [54]:
# Get the number of files in the other folders
other_apples_folder_paths = [
    "/content/drive/MyDrive/appel_crop_selectnew/Train/Rot_Apple",
    "/content/drive/MyDrive/appel_crop_selectnew/Train/Rot_Apple",
    "/content/drive/MyDrive/appel_crop_selectnew/Train/Blotch_Apple"
]

other_apples_count = 0

for folder_path in other_apples_folder_paths:
    other_apples_count += len(os.listdir(folder_path))

total_files = normal_apples_count + other_apples_count

percentage_normal = (normal_apples_count / total_files) * 100 if total_files > 0 else 0.0

print("Total Files:", total_files)
print("Normal Apples Count:", normal_apples_count)
print("Other Apples Count:", other_apples_count)
print("Percentage of Normal Apples:", percentage_normal)

Total Files: 419
Normal Apples Count: 75
Other Apples Count: 344
Percentage of Normal Apples: 17.899761336515514


In [55]:
# Chat loop
counter = 0

while counter < 10:
    # Generate a random number of not Normal Apples (x) and batch size (n)
    x = random.randint(20, 38)
    n = random.choice([20, 32])

    # Generate an input question about the AQL class using the random values
    input_question = f"The batch has {x} not Normal Apples. What is the AQL class if the batch size is {n}?"

    # Get the predicted AQL class using the text similarity model
    predicted_class = predict_aql_class(input_question)

    # Calculate the AQL class based on the random values
    calculated_class = calculate_aql_class(n, x)

    # Print the input question, predicted AQL class, and calculated AQL class
    print("Input Question:", input_question)
    print("Predicted AQL Class:", predicted_class)
    print("Calculated AQL Class:", calculated_class)
    print("-" * 50)
    counter += 1


Input Question: The batch has 32 not Normal Apples. What is the AQL class if the batch size is 20?
Predicted AQL Class: Unknown
Calculated AQL Class: class 4
--------------------------------------------------
Input Question: The batch has 21 not Normal Apples. What is the AQL class if the batch size is 32?
Predicted AQL Class: Unknown
Calculated AQL Class: Unknown
--------------------------------------------------
Input Question: The batch has 27 not Normal Apples. What is the AQL class if the batch size is 32?
Predicted AQL Class: Unknown
Calculated AQL Class: Unknown
--------------------------------------------------
Input Question: The batch has 26 not Normal Apples. What is the AQL class if the batch size is 32?
Predicted AQL Class: Unknown
Calculated AQL Class: Unknown
--------------------------------------------------
Input Question: The batch has 34 not Normal Apples. What is the AQL class if the batch size is 20?
Predicted AQL Class: Unknown
Calculated AQL Class: class 4
------